In [ ]:
import numpy as np
import torch
import torch.nn as nn
import sympy as sp
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torcheval.metrics.functional import r2_score
import architectures as archit
import data as da
import stable as sta

### graph and data


In [ ]:
num_vertices = 293 # number of vertices for two circulant graphs
not_moving_probabilities_vector = [0.05,0.05] # weight p for the diagonal element of graph 1 and 2
jump_sizes_vector = [1,30] # location of jump l
n_samples = 1000 # number of data pair

# generate the circulant graph tuple
ts = da.cycle_operator_tuple(num_vertices=num_vertices,
                    not_moving_probabilities_vector=not_moving_probabilities_vector,
                    jump_sizes_vector=jump_sizes_vector)
operator_tuple = ts

# normalized the graph tuple
ts_normalized = []
for ind_g, g in enumerate(ts):
  ts_normalized.append(g)
  if torch.linalg.matrix_norm(g,ord = 2) > 1:
    g_normalized = g / torch.linalg.matrix_norm(g,ord = 2)
    print("Graph {:d} has norm {:.4f} bigger than 1. This graph is normalized".format(ind_g,np.array(torch.linalg.matrix_norm(g,ord = 2))))
    ts_normalized[ind_g] = g_normalized
operator_tuple = (ts_normalized[0],ts_normalized[1])

# generate data using the circulant graph tuple
# y = 0.76*t1@t0@x + 0.33*t0@t1@x + 0.3*t0@t0@t0@x + noise
x, y = da.dataLab_cycles(num_vertices=num_vertices,
                      not_moving_probabilities_vector=not_moving_probabilities_vector,
                      jump_sizes_vector=jump_sizes_vector,
                      noise_stdev = 0.1,
                      n_samples = n_samples)

print(x.shape) # (number of data) * (number of features) * (number of veritces)
print(y.shape) # (number of data) * (number of features) * (number of veritces)

In [ ]:
# train test split
test_data_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_data_ratio)
print(X_train.shape)

In [ ]:
# use GPU if available
USE_GPU = True

if USE_GPU:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")

X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)
operator_tuple_device = (operator_tuple[0].to(device),operator_tuple[1].to(device))

### training function

In [ ]:
# perturbed graph tuple for stability metrics during training process
Z_tuple, Wj_minus_Zj_norm = sta.create_perturbation_Z(operator_tuple)
print(Wj_minus_Zj_norm)
Z_tuple_device = (Z_tuple[0].to(device),Z_tuple[1].to(device)) # perturbed graph tuple 


In [ ]:
def train_network(model, operator_tuple_device, Z_tuple_device, Wj_minus_Zj_norm, X_train, y_train, X_test, y_test, num_layer = 1, 
                  Penalty_lam = 0, Stable_Penalty = False, constrain_C_Flag = True, constrain_Cj_Flag = True, target_Upr_Cj_vec = 1, target_Upr_C = 1,
                  n_epochs = 5000, lr = 0.01, verbose = True, Plot_loss = True):

  num_vertices = X_train.shape[2]
  loss_fcn = nn.MSELoss()
  
  optimizer = torch.optim.Adam(model.parameters(), lr)

  epoch_tr_loss = []
  epoch_ts_loss = []
  h_poly_matrix_norm_all_GNN = []
  hw_minus_hz_poly_matrix_norm_all_GNN=[]
  hw_minus_hz_upper_all_GNN = []
  C_all_GNN = []
  C_j1_all_GNN = []
  C_j2_all_GNN = []
  for epoch in range(n_epochs):
    # training
    model.train()
    outs_train = model.forward(X_train)
    loss = loss_fcn(outs_train, y_train)

    # stability penalty
    if Stable_Penalty:
      loss = loss + Penalty_lam * sta.compute_penalty(model, target_Upr_Cj_vec, Upr_C = target_Upr_C,
                                                constrain_C_Flag = constrain_C_Flag, constrain_Cj_Flag = constrain_Cj_Flag)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_tr_loss.append(loss.item())
    y_train_reshape = torch.reshape(y_train,(y_train.shape[0],y_train.shape[1]*y_train.shape[2]))
    outs_train_reshape = torch.reshape(outs_train,(outs_train.shape[0],outs_train.shape[1]*outs_train.shape[2]))
    #R2Score
    R2_train = r2_score(outs_train_reshape, y_train_reshape)


    # testing
    model.eval()
    with torch.no_grad():
      outs_test = model.forward(X_test)
      y_test_reshape = torch.reshape(y_test,(y_test.shape[0],y_test.shape[1]*y_test.shape[2]))
      outs_test_reshape = torch.reshape(outs_test,(outs_test.shape[0],outs_test.shape[1]*outs_test.shape[2]))
      R2_test = r2_score(outs_test_reshape, y_test_reshape)
      GNN_test_loss = loss_fcn(outs_test, y_test)
      Penalty_test_loss = 0
      # stability penalty
      if Stable_Penalty:
        Penalty_test_loss = sta.compute_penalty(model, target_Upr_Cj_vec, Upr_C = target_Upr_C,
                                            constrain_C_Flag = constrain_C_Flag, constrain_Cj_Flag = constrain_Cj_Flag)
      Test_loss = GNN_test_loss + Penalty_lam * Penalty_test_loss
      epoch_ts_loss.append(Test_loss)

    # compute stability metrics for 1-layer model
    if num_layer == 1:
      h_poly_matrix_norm_all_GNN.append(sta.compute_h_poly_matrix_norm(model, num_vertices))
      hw_minus_hz_poly_matrix_norm_all_GNN.append(sta.compute_hw_minus_hz_poly_matrix_norm(model, num_vertices, operator_tuple_device, Z_tuple_device))

      C_max_sum_train_GNN, C_j_max_sum_train_GNN = sta.compute_constrain_param(model)
      C_max_sum_train_GNN = C_max_sum_train_GNN[0]
      C_j_max_sum_train_GNN = C_j_max_sum_train_GNN[0]
      C_all_GNN.append(C_max_sum_train_GNN.data)
      C_j1_all_GNN.append(C_j_max_sum_train_GNN[0].data)
      C_j2_all_GNN.append(C_j_max_sum_train_GNN[1].data)
      hw_minus_hz_upper_bound = C_j_max_sum_train_GNN.data.cpu() @ Wj_minus_Zj_norm
      hw_minus_hz_upper_all_GNN.append(hw_minus_hz_upper_bound)


    if verbose and (epoch % 10 == 0):
      print("Epoch {:05d} | Train Loss {:.4f} | Train_R^2 {:.4f} | Test Loss {:.4f} | Test_R^2 {:.4f} | Test GNN loss {:.4f} | Test penalty loss {:.4f} |".
            format(epoch,  epoch_tr_loss[epoch], R2_train, epoch_ts_loss[epoch], R2_test, GNN_test_loss, Penalty_test_loss))


  epoch_ts_loss = torch.stack(epoch_ts_loss).cpu()

  if Plot_loss:
    fig = plt.figure()
    epoch_seq=np.arange(100, len(epoch_tr_loss) + 1)
    plt.plot(epoch_seq, epoch_tr_loss[99:],'.-')
    plt.plot(epoch_seq, epoch_ts_loss[99:],'r.-')
    plt.grid()
    plt.xlabel('epochs')
    plt.ylabel('Loss')
    plt.title('Training Curve')
    plt.legend(['Training', 'Test'])
    plt.show()

  return R2_test, h_poly_matrix_norm_all_GNN, hw_minus_hz_poly_matrix_norm_all_GNN, hw_minus_hz_upper_all_GNN, C_all_GNN

### create monomial

In [ ]:
num_variable = 2
allowed_degree = 3
M = archit.MonomialWordSupport(num_variables=num_variable, allowed_degree = allowed_degree, device = device)

### 1-layer GtNN

In [ ]:
n_epochs=6000
num_features_in = 1
num_features_out = 1
Stable_Penalty = False
h_poly_matrix_norm_all_GNN = []
hw_minus_hz_poly_matrix_norm_all_GNN=[]
hw_minus_hz_upper_all_GNN = []
C_all_GNN = []

# evaluate monomial for training graph tuple
M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)

# 1-layer GtNN model
model_GNN = archit.OperatorFilterLayer(num_features_in = num_features_in,
                                            num_features_out = num_features_out, monomial_word_support = M)
model_GNN.to(device)

# training
result_GNN, h_poly_matrix_norm_all_GNN, hw_minus_hz_poly_matrix_norm_all_GNN, hw_minus_hz_upper_all_GNN, C_all_GNN = train_network(
    model_GNN, operator_tuple_device, Z_tuple_device, Wj_minus_Zj_norm, X_train, y_train, X_test, y_test, 
    Stable_Penalty = Stable_Penalty, n_epochs = n_epochs)

In [ ]:
# final testing R^2
print(result_GNN)

In [ ]:
# plot 1-layer GtNN stability metrics
h_poly_matrix_norm_all_GNN = torch.stack(h_poly_matrix_norm_all_GNN).cpu()
C_all_GNN = torch.stack(C_all_GNN).cpu()
hw_minus_hz_poly_matrix_norm_all_GNN = torch.stack(hw_minus_hz_poly_matrix_norm_all_GNN).cpu()
hw_minus_hz_upper_all_GNN = torch.stack(hw_minus_hz_upper_all_GNN).cpu()


fig = plt.figure()
epoch_seq2 = np.arange(0, len(h_poly_matrix_norm_all_GNN) )
plt.rc('font',size=15)
plt.plot(epoch_seq2,h_poly_matrix_norm_all_GNN,'-',label='operator norm of h(T)')
plt.plot(epoch_seq2,C_all_GNN,'-', label = 'C(h)')
plt.xlabel('epochs')
plt.ylabel('stability metric')
plt.title('GtNN')
plt.legend()
plt.show()

fig = plt.figure()
plt.rc('font',size=15)
plt.plot(epoch_seq2,hw_minus_hz_poly_matrix_norm_all_GNN,'-',label='operator norm of h(W) - h(T)')
plt.plot(epoch_seq2,hw_minus_hz_upper_all_GNN,'-', label = 'upper bound on operator norm of h(W) - h(T)' )
plt.xlabel('epochs')
plt.ylabel('stability metric')
plt.title('GtNN')
plt.legend(fontsize=12)
plt.show()


In [ ]:
# save data
torch.save(h_poly_matrix_norm_all_GNN, 'h_poly_matrix_norm_all_GNN.pt')
torch.save(C_all_GNN, 'C_all_GNN.pt')
torch.save(hw_minus_hz_poly_matrix_norm_all_GNN, 'hw_minus_hz_poly_matrix_norm_all_GNN.pt')
torch.save(hw_minus_hz_upper_all_GNN, 'hw_minus_hz_upper_all_GNN.pt')

### 1-layer stable GtNN

In [ ]:
# Compute expansion constant for 1-layer GtNN
C_max_sum, C_j_max_sum = sta.compute_constrain_param(model_GNN)
print(C_max_sum)
print(C_j_max_sum)

# Set the target expansion constant for stable GtNN as 1/2 of the GtNN
target_Upr_C = []
target_Upr_Cj_vec = []
for C_max_sum_each_layer in C_max_sum:
  target_Upr_C.append(C_max_sum_each_layer.data/2)

for C_j_max_sum_each_layer in C_j_max_sum:
  target_Upr_Cj_vec.append(C_j_max_sum_each_layer.data/2)

print(target_Upr_C)
print(target_Upr_Cj_vec)

C_max_sum = C_max_sum[0]
C_j_max_sum = C_j_max_sum[0]

In [ ]:
n_epochs=6000
Penalty_lam = 10 # penatly coefficient
Stable_Penalty = True
h_poly_matrix_norm_all_stable = []
hw_minus_hz_poly_matrix_norm_all_stable=[]
hw_minus_hz_upper_all_stable = []
C_all_stable = []

# evaluate monomial for training graph tuple
M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)

# 1-layer stable GtNN model
model_stable = archit.OperatorFilterLayer(num_features_in = num_features_in,
                                            num_features_out = num_features_out, monomial_word_support = M)
model_stable.to(device)

# training
result_stable, h_poly_matrix_norm_all_stable, hw_minus_hz_poly_matrix_norm_all_stable, hw_minus_hz_upper_all_stable, C_all_stable = train_network(
    model_stable, operator_tuple_device, Z_tuple_device, Wj_minus_Zj_norm, X_train, y_train, X_test, y_test, 
    Penalty_lam = Penalty_lam, Stable_Penalty = Stable_Penalty, target_Upr_Cj_vec = target_Upr_Cj_vec, target_Upr_C = target_Upr_C,
    n_epochs = n_epochs)

In [ ]:
# final testing R^2
print(result_stable)

In [ ]:
# plot 1-layer stable GtNN stability metrics
h_poly_matrix_norm_all_stable = torch.stack(h_poly_matrix_norm_all_stable).cpu()
C_all_stable = torch.stack(C_all_stable).cpu()
hw_minus_hz_poly_matrix_norm_all_stable = torch.stack(hw_minus_hz_poly_matrix_norm_all_stable).cpu()
hw_minus_hz_upper_all_stable = torch.stack(hw_minus_hz_upper_all_stable).cpu()

fig = plt.figure()
epoch_seq_stable2 = np.arange(0, len(h_poly_matrix_norm_all_stable) )
plt.rc('font',size=15)
plt.plot(epoch_seq_stable2,h_poly_matrix_norm_all_stable,'-',label='operator norm of h(T)', alpha =0.7)
plt.plot(epoch_seq_stable2,C_all_stable,'-', label = 'C(h)', alpha =0.7)
plt.xlabel('epochs')
plt.ylabel('stability metric')
plt.title('stable GtNN')
plt.legend()
plt.show()

fig = plt.figure()
plt.plot(epoch_seq_stable2,hw_minus_hz_poly_matrix_norm_all_stable,'-',label='operator norm of h(W) - h(T)', alpha =0.7)
plt.plot(epoch_seq_stable2,hw_minus_hz_upper_all_stable,'-', label = 'upper bound on operator norm of h(W) - h(T)', alpha =0.7)
plt.rc('font',size=15)
plt.xlabel('epochs')
plt.ylabel('stability metric')
plt.title('stable GtNN')
plt.legend(fontsize = 12)
plt.show()

In [ ]:
torch.save(h_poly_matrix_norm_all_stable, 'h_poly_matrix_norm_all_stable.pt')
torch.save(C_all_stable, 'C_all_stable.pt')
torch.save(hw_minus_hz_poly_matrix_norm_all_stable, 'hw_minus_hz_poly_matrix_norm_all_stable.pt')
torch.save(hw_minus_hz_upper_all_stable, 'hw_minus_hz_upper_all_stable.pt')

In [ ]:
# print expanion constant to check if satisfy constraints
C_max_sum_stable, C_j_max_sum_stable = sta.compute_constrain_param(model_stable)
C_max_sum_stable = C_max_sum_stable[0]
C_j_max_sum_stable = C_j_max_sum_stable[0]
print(C_max_sum_stable)
print(C_j_max_sum_stable)

### plot figure

In [ ]:
# combine graph for stability metric for GtNN and stabe GtNN (if same number of epochs)
fig = plt.figure()
epoch_seq_stable2 = np.arange(0, len(h_poly_matrix_norm_all_stable) )
plt.rc('font',size=20)
plt.plot(epoch_seq_stable2,C_all_GNN,'--', color= 'tab:orange', label = 'GtNN: $C(h)$', alpha =0.7)
plt.plot(epoch_seq_stable2,h_poly_matrix_norm_all_GNN,'-',color= 'tab:orange', label='GtNN: $\|h(T)\|_{op}$', alpha =0.7)
plt.plot(epoch_seq_stable2,C_all_stable,'--',  color= 'tab:blue', label = 'stable GtNN: $C(h)$', alpha =0.7)
plt.plot(epoch_seq_stable2,h_poly_matrix_norm_all_stable,'-', color= 'tab:blue',label='stable GtNN: $\|h(T)\|_{op}$', alpha =0.7)
plt.xlabel('epochs')
plt.ylabel('stability metric')

plt.legend()
plt.show()

fig = plt.figure()
plt.rc('font',size=20)
plt.plot(epoch_seq_stable2,hw_minus_hz_upper_all_GNN,'--', color= 'tab:orange', label = 'GtNN: $\|h(W) - h(T)\|_{op}$ bound', alpha =0.7)
plt.plot(epoch_seq_stable2,hw_minus_hz_poly_matrix_norm_all_GNN,'-', color= 'tab:orange',label='GtNN: $\|h(W) - h(T)\|_{op}$', alpha =0.7)
plt.plot(epoch_seq_stable2,hw_minus_hz_upper_all_stable,'--',color= 'tab:blue', label = 'stable GtNN: $\|h(W) - h(T)\|_{op}$ bound', alpha =0.7)
plt.plot(epoch_seq_stable2,hw_minus_hz_poly_matrix_norm_all_stable,'-',color= 'tab:blue',label='stable GtNN: $\|h(W) - h(T)\|_{op}$', alpha =0.7)
plt.xlabel('epochs',fontsize = 20)
plt.ylabel('stability metric', fontsize = 20)
plt.legend(fontsize=15)
plt.show()

### 2-layer GtNN

In [ ]:
n_epochs=8000
num_features_in = 1
num_features_out = 1
num_features_hidden = 2
num_layer = 2
Stable_Penalty = False
h_poly_matrix_norm_all_twolayer = []
hw_minus_hz_poly_matrix_norm_all_twolayer=[]
hw_minus_hz_upper_all_twolayer = []
C_all_twolayer = []

# evaluate monomial for training graph tuple
M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)

# 2-layer GtNN model
model_two_layer_GNN = archit.TwoLayerWithReLU(num_features_in = num_features_in, num_features_out = num_features_out, 
                                              num_features_hidden = num_features_hidden, monomial_word_support = M)
model_two_layer_GNN.to(device)

# training
result_twolayer, h_poly_matrix_norm_all_twolayer, hw_minus_hz_poly_matrix_norm_all_twolayer, hw_minus_hz_upper_all_twolayer, C_all_twolayer = train_network(
    model_two_layer_GNN, operator_tuple_device, Z_tuple_device, Wj_minus_Zj_norm, X_train, y_train, X_test, y_test, num_layer = num_layer,
    Stable_Penalty = Stable_Penalty, n_epochs = n_epochs)

In [ ]:
# final testing R^2
print(result_twolayer)

### 2-layer GtNN

In [ ]:
# Compute expansion constant for 2-layer GtNN
[C_max_sum_all_layer, C_j_max_sum_all_layer] = sta.compute_constrain_param(model_two_layer_GNN)
print(C_max_sum_all_layer)
print(C_j_max_sum_all_layer)

# Set the target expansion constant for 2-layer stable GtNN as 1/2 of the 2-layer GtNN
target_Upr_C_all_layer = []
target_Upr_Cj_all_layer = []
for C_max_sum_each_layer in C_max_sum_all_layer:
  target_Upr_C_all_layer.append(C_max_sum_each_layer.data/2)

for C_j_max_sum_each_layer in C_j_max_sum_all_layer:
  target_Upr_Cj_all_layer.append(C_j_max_sum_each_layer.data/2)

print(target_Upr_C_all_layer)
print(target_Upr_Cj_all_layer)

In [ ]:
n_epochs=8000
Penalty_lam = 10 # penatly coefficient
Stable_Penalty = True
h_poly_matrix_norm_all_twolayer_stable = []
hw_minus_hz_poly_matrix_norm_all_twolayer_stable=[]
hw_minus_hz_upper_all_twolayer_stable = []
C_all_twolayer_stable = []

# evaluate monomial for training graph tuple
M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)

# 2-layer stable GtNN model
model_two_layer_stable = archit.TwoLayerWithReLU(num_features_in = num_features_in, num_features_out = num_features_out, 
                                              num_features_hidden = num_features_hidden, monomial_word_support = M)
model_two_layer_stable.to(device)

# training
result_twolayer_stable, h_poly_matrix_norm_all_twolayer_stable, hw_minus_hz_poly_matrix_norm_all_twolayer_stable, hw_minus_hz_upper_all_twolayer_stable, C_all_twolayer_stable = train_network(
    model_two_layer_stable, operator_tuple_device, Z_tuple_device, Wj_minus_Zj_norm, X_train, y_train, X_test, y_test, num_layer = num_layer,
    Penalty_lam = Penalty_lam, Stable_Penalty = Stable_Penalty, target_Upr_Cj_vec = target_Upr_Cj_all_layer, target_Upr_C = target_Upr_C_all_layer,
    n_epochs = n_epochs)

In [ ]:
# final testing R^2
print(result_twolayer_stable)

In [ ]:
# print expanion constant to check if satisfy constraints
[C_max_sum_two_layer_stable, C_j_max_sum_two_layer_stable] = sta.compute_constrain_param(model_two_layer_stable)
print(C_max_sum_two_layer_stable)
print(C_j_max_sum_two_layer_stable)

### output perturbation for different graph perturbation

In [ ]:
Perturbation_norm_all = np.arange(0,5,0.5) # initial peturbation size
num_perturb_norm = len(Perturbation_norm_all) # number of different size of peturbation
num_perturb = 10 # number of repeated peturbation for each size

out_perturb_GNN_all = np.zeros((num_perturb,num_perturb_norm)) # output perturbation for 1-layer GtNN
out_perturb_stable_all = np.zeros((num_perturb,num_perturb_norm)) # output perturbation for 1-layer stable GtNN
out_perturb_two_layer_GNN_all = np.zeros((num_perturb,num_perturb_norm)) # output perturbation for 2-layer GtNN
out_perturb_two_layer_stable_all = np.zeros((num_perturb,num_perturb_norm)) # output perturbation for 2-layer stable GtNN

diff_out_perturb = np.zeros((num_perturb,num_perturb_norm))
relative_diff_out = np.zeros((num_perturb,num_perturb_norm))

# R^2 on testing graph
R2_test_GNN = np.zeros((num_perturb,num_perturb_norm))
R2_test_hat_GNN = np.zeros((num_perturb,num_perturb_norm))
R2_test_stable = np.zeros((num_perturb,num_perturb_norm))
R2_test_hat_stable = np.zeros((num_perturb,num_perturb_norm))
R2_test_two_layer_GNN = np.zeros((num_perturb,num_perturb_norm))
R2_test_hat_two_layer_GNN = np.zeros((num_perturb,num_perturb_norm))
R2_test_two_layer_stable = np.zeros((num_perturb,num_perturb_norm))
R2_test_hat_two_layer_stable = np.zeros((num_perturb,num_perturb_norm))

# actual peturbation size after normalizaion
actual_peturbation_size = np.zeros((num_perturb,num_perturb_norm,2))

for ind_perturb, trainStabilityEpsilon in enumerate(Perturbation_norm_all):
  for i in range(num_perturb):

    # create perturbed graph tuple
    ts_perturb = []
    for ind_g, g in enumerate(operator_tuple):
      actual_peturbation_size[i,ind_perturb,ind_g] = trainStabilityEpsilon
      if trainStabilityEpsilon > 0:
        S = g
        E = torch.rand(size=S.shape)
        E = torch.triu(E) + torch.triu(E,diagonal = 1).t()
        E = trainStabilityEpsilon * E / torch.linalg.matrix_norm(E,ord = 2)
        S_hat= S + E # additive perturbation
      else:
        S_hat = g

      # normalized perturbed graph 
      if torch.linalg.matrix_norm(S_hat,ord = 2) > 1:
        S_hat = S_hat / torch.linalg.matrix_norm(S_hat,ord = 2)
        E_actual = S_hat - S
        # compute actual peturbation size after normalizaion
        trainStabilityEpsilon_actual = torch.linalg.matrix_norm(E_actual,ord = 2)
        actual_peturbation_size[i,ind_perturb,ind_g] = trainStabilityEpsilon_actual
      ts_perturb.append(S_hat)
    ts_perturb_tuple = (ts_perturb[0],ts_perturb[1])
    ts_perturb_tuple_device = (ts_perturb[0].to(device),ts_perturb[1].to(device))

    # evaluate stability performance for 1-layer GtNN
    model_GNN.eval()
    with torch.no_grad():
      # evaluate on original graph tuple
      M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
      model_GNN.change_monomial_word_support(M)
      outs_test = model_GNN.forward(X_test)

      # compute testing R^2 for original graph tuple
      y_test_reshape = torch.reshape(y_test,(y_test.shape[0],y_test.shape[1]*y_test.shape[2]))
      outs_test_reshape = torch.reshape(outs_test,(outs_test.shape[0],outs_test.shape[1]*outs_test.shape[2]))
      R2_test_GNN[i,ind_perturb] = r2_score(outs_test_reshape, y_test_reshape)

      # evaluate on perturbed graph tuple
      M.evaluate_at_operator_tuple(operator_tuple = ts_perturb_tuple_device)
      model_GNN.change_monomial_word_support(M)
      outs_test_hat = model_GNN.forward(X_test)

      #compute testing R^2 for perturbed graph tuple
      outs_test_hat_reshape = torch.reshape(outs_test_hat,(outs_test_hat.shape[0],outs_test_hat.shape[1]*outs_test_hat.shape[2]))
      R2_test_hat_GNN[i,ind_perturb] = r2_score(outs_test_hat_reshape, y_test_reshape)

      # compute output perturbation
      out_perturb_GNN = torch.norm(outs_test - outs_test_hat)
      print("GtNN out_perturb for %d:"%i)
      print(out_perturb_GNN.cpu().numpy())

    # evaluate stability performance for 1-layer stable GtNN
    model_stable.eval()
    with torch.no_grad():
      # evaluate on original graph tuple
      M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
      model_stable.change_monomial_word_support(M)
      outs_test_stable = model_stable.forward(X_test)

      # compute testing R^2 for original graph tuple
      y_test_reshape = torch.reshape(y_test,(y_test.shape[0],y_test.shape[1]*y_test.shape[2]))
      outs_test_stable_reshape = torch.reshape(outs_test_stable,(outs_test_stable.shape[0],outs_test_stable.shape[1]*outs_test_stable.shape[2]))
      R2_test_stable[i,ind_perturb] = r2_score(outs_test_stable_reshape, y_test_reshape)

      # evaluate on perturbed graph tuple
      M.evaluate_at_operator_tuple(operator_tuple = ts_perturb_tuple_device)
      model_stable.change_monomial_word_support(M)
      outs_test_stable_hat = model_stable.forward(X_test)

      #compute testing R^2 for perturbed graph tuple
      outs_test_hat_stable_reshape = torch.reshape(outs_test_stable_hat,(outs_test_stable_hat.shape[0],outs_test_stable_hat.shape[1]*outs_test_stable_hat.shape[2]))
      R2_test_hat_stable[i,ind_perturb] = r2_score(outs_test_hat_stable_reshape, y_test_reshape)

      # compute output perturbation
      out_perturb_Stable = torch.norm(outs_test_stable - outs_test_stable_hat)
      print("stable GtNN out_perturb for %d:"%i)
      print(out_perturb_Stable.cpu().numpy())

    # evaluate stability performance for 2-layer GtNN
    model_two_layer_GNN.eval()
    with torch.no_grad():
      #evaluate on original graph tuple
      M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
      model_two_layer_GNN.change_monomial_word_support(M)
      outs_test_two_layer_GNN = model_two_layer_GNN.forward(X_test)

      # compute testing R^2 for original graph tuple
      y_test_reshape = torch.reshape(y_test,(y_test.shape[0],y_test.shape[1]*y_test.shape[2]))
      outs_test_two_layer_GNN_reshape = torch.reshape(outs_test_two_layer_GNN,(outs_test_two_layer_GNN.shape[0],outs_test_two_layer_GNN.shape[1]*outs_test_two_layer_GNN.shape[2]))
      R2_test_two_layer_GNN[i,ind_perturb] = r2_score(outs_test_two_layer_GNN_reshape, y_test_reshape)

      # evaluate on perturbed graph tuple
      M.evaluate_at_operator_tuple(operator_tuple = ts_perturb_tuple_device)
      model_two_layer_GNN.change_monomial_word_support(M)
      outs_test_two_layer_GNN_hat = model_two_layer_GNN.forward(X_test)

      #compute testing R^2 for perturbed graph tuple
      outs_test_hat_two_layer_GNN_reshape = torch.reshape(outs_test_two_layer_GNN_hat,(outs_test_two_layer_GNN_hat.shape[0],outs_test_two_layer_GNN_hat.shape[1]*outs_test_two_layer_GNN_hat.shape[2]))
      R2_test_hat_two_layer_GNN[i,ind_perturb] = r2_score(outs_test_hat_two_layer_GNN_reshape, y_test_reshape)

      # compute output perturbation
      out_perturb_two_layer_GNN = torch.norm(outs_test_two_layer_GNN - outs_test_two_layer_GNN_hat)
      print("two layer GtNN out_perturb for %d:"%i)
      print(out_perturb_two_layer_GNN.cpu().numpy())

    # evaluate stability performance for 2-layer stable GtNN
    model_two_layer_stable.eval()
    with torch.no_grad():
      #evaluate on original graph tuple
      M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
      model_two_layer_stable.change_monomial_word_support(M)
      outs_test_two_layer_stable = model_two_layer_stable.forward(X_test)

      # compute testing R^2 for original graph tuple
      y_test_reshape = torch.reshape(y_test,(y_test.shape[0],y_test.shape[1]*y_test.shape[2]))
      outs_test_two_layer_stable_reshape = torch.reshape(outs_test_two_layer_stable,(outs_test_two_layer_stable.shape[0],outs_test_two_layer_stable.shape[1]*outs_test_two_layer_stable.shape[2]))
      R2_test_two_layer_stable[i,ind_perturb] = r2_score(outs_test_two_layer_stable_reshape, y_test_reshape)

      # evaluate on perturbed graph tuple
      M.evaluate_at_operator_tuple(operator_tuple = ts_perturb_tuple_device)
      model_two_layer_stable.change_monomial_word_support(M)
      outs_test_two_layer_stable_hat = model_two_layer_stable.forward(X_test)

      #compute testing R^2 for perturbed graph tuple
      outs_test_hat_two_layer_stable_reshape = torch.reshape(outs_test_two_layer_stable_hat,(outs_test_two_layer_stable_hat.shape[0],outs_test_two_layer_stable_hat.shape[1]*outs_test_two_layer_stable_hat.shape[2]))
      R2_test_hat_two_layer_stable[i,ind_perturb] = r2_score(outs_test_hat_two_layer_stable_reshape, y_test_reshape)

      # compute output perturbation
      out_perturb_two_layer_stable = torch.norm(outs_test_two_layer_stable - outs_test_two_layer_stable_hat)
      print("two layer stable GtNN out_perturb for %d:"%i)
      print(out_perturb_two_layer_stable.cpu().numpy())

    out_perturb_GNN_all[i,ind_perturb] = out_perturb_GNN.cpu().numpy()
    out_perturb_stable_all[i,ind_perturb] = out_perturb_Stable.cpu().numpy()
    out_perturb_two_layer_GNN_all[i,ind_perturb] = out_perturb_two_layer_GNN.cpu().numpy()
    out_perturb_two_layer_stable_all[i,ind_perturb] = out_perturb_two_layer_stable.cpu().numpy()
    diff_out_perturb[i,ind_perturb] = out_perturb_GNN.cpu().numpy() - out_perturb_Stable.cpu().numpy()
    relative_diff_out[i,ind_perturb] = diff_out_perturb[i,ind_perturb] / out_perturb_Stable.cpu().numpy()



In [ ]:
# R^2 on perturbed graph tuple
actual_peturbation_size_avg_j = np.mean(actual_peturbation_size,axis = 2)
plt.scatter(actual_peturbation_size_avg_j, R2_test_hat_GNN,marker='.',label='GtNN')
plt.scatter(actual_peturbation_size_avg_j,R2_test_hat_stable,marker='.',label='stable GtNN')
plt.scatter(actual_peturbation_size_avg_j, R2_test_hat_two_layer_GNN,marker='.',label='two layer GtNN')
plt.scatter(actual_peturbation_size_avg_j,R2_test_hat_two_layer_stable,marker='.',label='two layer stable GtNN')
plt.legend()
plt.xlabel("size of graph perturbation")
plt.ylabel("R square on perturbed graph")
plt.show()

In [ ]:
# output perturbation figure
actual_peturbation_size_avg_j = np.mean(actual_peturbation_size,axis = 2)
plt.rc('font',size=20)
plt.scatter(actual_peturbation_size_avg_j, out_perturb_GNN_all,label='GtNN',marker='.', alpha = 1)
plt.scatter(actual_peturbation_size_avg_j,out_perturb_stable_all,label='stable GtNN',marker='.', alpha = 1)
plt.scatter(actual_peturbation_size_avg_j, out_perturb_two_layer_GNN_all,label='two layer GtNN', marker='.', alpha = 1)
plt.scatter(actual_peturbation_size_avg_j,out_perturb_two_layer_stable_all,label='two layer stable GtNN',marker='.', alpha = 1)
plt.legend(fontsize=15)
plt.xlabel("size of graph perturbation")
plt.ylabel("output perturbation")
plt.show()

In [ ]:
# save data
with open('peturbation.npy', 'wb') as f:
    np.save(f,actual_peturbation_size_avg_j)
    np.save(f,out_perturb_GNN_all)
    np.save(f,out_perturb_stable_all)
    np.save(f,out_perturb_two_layer_GNN_all)
    np.save(f,out_perturb_two_layer_stable_all)

### output perturbation for different input node features perturbation

In [ ]:
Perturbation_feature_all = np.arange(0,10,0.1) # peturbation size
num_perturb_feature = len(Perturbation_feature_all) # number of perturbation size

out_perturb_feature_GNN_all = np.zeros((X_test.shape[0], num_perturb_feature)) # output perturbation for 1-layer GtNN
out_perturb_feature_stable_all = np.zeros((X_test.shape[0], num_perturb_feature)) # output perturbation for 1-layer stable GtNN
out_perturb_feature_two_layer_GNN_all = np.zeros((X_test.shape[0], num_perturb_feature)) # output perturbation for 2-layer GtNN
out_perturb_feature_two_layer_stable_all = np.zeros((X_test.shape[0], num_perturb_feature)) # output perturbation for 2-layer stable GtNN

for ind_perturb, trainStabilityEpsilon in enumerate(Perturbation_feature_all):
  # create perturbed input
  E_feature = torch.rand(size=X_test.shape)
  E_feature = E_feature.to(device)
  E_feature_norm_each_data = torch.max(torch.linalg.vector_norm(E_feature,ord = 2,dim = 2),dim=1)[0]
  E_feature_fix_size = trainStabilityEpsilon * E_feature / E_feature_norm_each_data[:, None, None]
  X_test_hat = X_test + E_feature_fix_size

  # evaluate stability performance for 1-layer GtNN
  model_GNN.eval()
  with torch.no_grad():
    M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
    model_GNN.change_monomial_word_support(M)
    outs_test = model_GNN.forward(X_test)
    outs_test_hat = model_GNN.forward(X_test_hat)
    # output perturbation
    out_perturb_feature_GNN = torch.max(torch.linalg.vector_norm(outs_test - outs_test_hat,ord = 2,dim = 2),dim=1)[0]
    print("average GNN feature out_perturb: ")
    print(np.mean(out_perturb_feature_GNN.cpu().numpy()))

  # evaluate stability performance for 1-layer stable GtNN
  model_stable.eval()
  with torch.no_grad():
    M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
    model_stable.change_monomial_word_support(M)
    outs_test_stable = model_stable.forward(X_test)
    outs_test_stable_hat = model_stable.forward(X_test_hat)
    # output perturbation
    out_perturb_feature_Stable = torch.max(torch.linalg.vector_norm(outs_test_stable - outs_test_stable_hat,ord = 2,dim = 2),dim=1)[0]
    print("average stable GNN feature out_perturb: ")
    print(np.mean(out_perturb_feature_Stable.cpu().numpy()))

  # evaluate stability performance for 2-layer GtNN
  model_two_layer_GNN.eval()
  with torch.no_grad():
    M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
    model_two_layer_GNN.change_monomial_word_support(M)
    outs_test_two_layer_GNN = model_two_layer_GNN.forward(X_test)
    outs_test_two_layer_GNN_hat = model_two_layer_GNN.forward(X_test_hat)
    # output perturbation
    out_perturb_feature_two_layer_GNN = torch.max(torch.linalg.vector_norm(outs_test_two_layer_GNN - outs_test_two_layer_GNN_hat,ord = 2,dim = 2),dim=1)[0]
    print("average two layer GNN feature out_perturb: ")
    print(np.mean(out_perturb_feature_two_layer_GNN.cpu().numpy()))

  # evaluate stability performance for 2-layer stable GtNN
  model_two_layer_stable.eval()
  with torch.no_grad():
    M.evaluate_at_operator_tuple(operator_tuple=operator_tuple_device)
    model_two_layer_stable.change_monomial_word_support(M)
    outs_test_two_layer_stable = model_two_layer_stable.forward(X_test)
    outs_test_two_layer_stable_hat = model_two_layer_stable.forward(X_test_hat)
    # output perturbation
    out_perturb_feature_two_layer_stable = torch.max(torch.linalg.vector_norm(outs_test_two_layer_stable - outs_test_two_layer_stable_hat,ord = 2,dim = 2),dim=1)[0]
    print("average two layer stable GNN feature out_perturb: ")
    print(np.mean(out_perturb_feature_two_layer_stable.cpu().numpy()))

  out_perturb_feature_GNN_all[:,ind_perturb] = out_perturb_feature_GNN.cpu().numpy()
  out_perturb_feature_stable_all[:,ind_perturb] = out_perturb_feature_Stable.cpu().numpy()
  out_perturb_feature_two_layer_GNN_all[:,ind_perturb] = out_perturb_feature_two_layer_GNN.cpu().numpy()
  out_perturb_feature_two_layer_stable_all[:,ind_perturb] = out_perturb_feature_two_layer_stable.cpu().numpy()


In [ ]:
# output perturbation figure
fig = plt.figure()
plt.rc('font',size=20)
plt.scatter(Perturbation_feature_all, np.mean(out_perturb_feature_GNN_all, axis=0), marker='+', label='GtNN', alpha = 0.5)
plt.scatter(Perturbation_feature_all, np.mean(out_perturb_feature_stable_all, axis = 0), marker='x', label='stable GtNN', alpha = 0.5)
plt.scatter(Perturbation_feature_all, np.mean(out_perturb_feature_two_layer_GNN_all, axis=0), marker='1', label='two layer GtNN', alpha = 0.5)
plt.scatter(Perturbation_feature_all, np.mean(out_perturb_feature_two_layer_stable_all, axis=0), marker='.',label='two layer stable GtNN', alpha = 0.5)

plt.legend(fontsize = 15)
plt.xlabel("size of feature perturbation")
plt.ylabel("output perturbation")
plt.show()

In [ ]:
with open('peturbation_feature.npy', 'wb') as f:
    np.save(f,Perturbation_feature_all)
    np.save(f,out_perturb_feature_GNN_all)
    np.save(f,out_perturb_feature_stable_all)
    np.save(f,out_perturb_feature_two_layer_GNN_all)
    np.save(f,out_perturb_feature_two_layer_stable_all)